In [1]:
import pandas

In [10]:
class MinoRepo():
    def __init__(self,fact_table):
        self.fact_table = fact_table
        self.fields = fact_table.columns.tolist()
        self.fact_fields = fact_table.columns.tolist()
        self.dim_fields = {}
        self.foreignKeys ={}
        self.tables = [fact_table]
        self.master_tables = {}
        self.master_tables_names = []
        
    def create_dim(self, field_to_move, dim_table_name):
        __subtable__ = self.fact_table[field_to_move]
        __aux_unique_values = __subtable__.unique().tolist()
        foreing_key = self.__get__next_foreign_key__()
        self.fact_table[foreing_key] = self.fact_table[field_to_move].map(lambda x:__aux_unique_values.index(x))
        self.fact_table = ta.drop(field_to_move,axis = 1)
        __temp_dim_table__ = pandas.DataFrame(__aux_unique_values,columns = [field_to_move])
        __temp_dim_table__[foreing_key] = __temp_dim_table__.index
        __temp_dim_table__.set_index(foreing_key)
#        setattr(self, dim_table_name, __temp_dim_table__)
        
        #Adding to lists
        self.master_tables[dim_table_name] = __temp_dim_table__
        self.master_tables_names.append(dim_table_name)
        
        if len(self.foreignKeys) == 0:
            self.foreignKeys[foreing_key] = [dim_table_name]
        elif len(self.foreignKeys[foreing_key]) :
            self.foreignKeys[foreing_key].append(dim_table_name)
        else:
            self.foreignKeys[foreing_key] = [dim_table_name]

        
        #field lists
        self.dim_fields[field_to_move] = dim_table_name
        self.fact_fields.remove(field_to_move)
        
        
    def __get__next_foreign_key__(self):
        __list__ = list(self.foreignKeys.keys())
        prefix_foreing_key = '__FK__'
        if len(__list__) > 0:
            __list__.sort(reverse=True)
            __last_key__ = __list__[0]
            foreing_key_next = __last_key__.replace(prefix_foreing_key,'')
            if foreing_key_next.isnumeric():
                foreing_key_next = int(foreing_key_next)+1
                return prefix_foreing_key+str(prefix_foreing_key).zfill(2)
        foreing_key_next = 1
        return prefix_foreing_key+str(foreing_key_next).zfill(2)
    
    def filter_facts(self, filter_value, filter_field):
        if filter_field.__class__ == list:
            print('Function not yet avalaible, please filter a single dim')
        return self.__filter_one_dim__(filter_value,filter_field)
    
    def __filter_one_dim__(self,filter_value,filter_field):
        filter_table_name = self.dim_fields[filter_field]
        filter_table = bb.master_tables[filter_table_name]
        #We extract the foreign keys of the table
        #First we get the names
        _foreing_keys_list = []
        for __colName in filter_table.columns.tolist():
            if __colName.count('__FK__')>0:
                _foreing_keys_list.append(__colName)
        _foreing_keys = ' , '.join(_foreing_keys_list)
        #Now we get the submatix that has the foreign keys as columns and the rows of the value we got
        index_filter = filter_table.loc[filter_table[filter_field] == filter_value,[_foreing_keys]].index
        filtered_list = []
        for _idx in index_filter:
            __row = {}
            for __colName in _foreing_keys_list:
                __row[__colName] = self.master_tables[filter_table_name].ix[_idx,__colName]
            filtered_list.append(__row)
        #Now we create the query
        filter_query_list = []
        for __row in filtered_list:
            query_as_list = []
        
            for __colName  in __row.keys():
                column_query = 'self.fact_table["'+__colName+'"] == {0}'.format(str(__row[__colName]))
                query_as_list.append(column_query)
            
            filter_query = '(' +  ') && ('.join(query_as_list) + ')'
            filter_query_list.append(filter_query)
        
        final_filter_query = '(' + ' | '.join(filter_query_list) + ')'
            
        fields_to_load = chr(34) + '" , "'.join(bb.fact_fields) + chr(34)
        return self.fact_table.loc[eval(final_filter_query),eval(fields_to_load)]

In [11]:
ta = pandas.read_csv('cups_012016.txt',delimiter = ';')
bb = MinoRepo(ta)

In [12]:
bb.create_dim('name','dim_name')

In [13]:
bb.filter_facts('ES0169000000490350LW0F','name')

KeyError: 'the label [((self.fact_table["__FK__01"] == 18))] is not in the [index]'